<a href="https://colab.research.google.com/github/super-onigiri-man/ITE3-2/blob/main/FM_%E3%83%99%E3%82%B9%E3%83%88%E3%83%92%E3%83%83%E3%83%88%E3%83%A9%E3%83%B3%E3%82%AD%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**FM愛媛 ベストヒットランキング ランキング早見表の使い方**



1.   左上のOpen in Colabを押してGoogle Colabを起動させます
2.   右のタブから明屋書店のデータを入れます。

　　　（このページを閉じるとこのページにある明屋書店のデータも消えます）
3.   コード左上の再生ボタン「上から」押す（コードが実行されます）
4.   途中で見たい週を選択します（1:今週,2:先週,3:選択した週）

5
.   1,2の場合は約30秒で結果が出力されます。

     （年末年始明けなどのランキングの日付が特殊な場合は3を選択してください）

     （水曜日の14時以降にランキングがサイトに掲載されるため、水曜日の正午以前に1を選択するとエラーが出ます）

     （日付がたまにバグるときがあります。日付を確認してください）

6
.   3の場合はYYYY-MM-DDの順序で入力してください。

     （例:2023年11月28日→2023-11-28）

     （これを選択した場合、入力された一番近い発表日のランキングが出ます）

     （未来の日付を入力するとエラーになります）

    

**利用上の注意！**

・このコードを書き換えないでください。書き換えると動かなくなる可能性があります。

  （将来の僕より優秀なプログラマーのADの方、このクソコードのおリファクタリングよろしくお願いします）

・連続で使用する場合は最低でも3分以上時間を開けてください。

  （連続でアクセスすると各サイトのサーバーが攻撃とみなし、アクセス制限をかけられる場合があります。）

・このシステムはリンクや明屋書店のExcelのフォーマットが変わると動かなくなります。

勉強になるかわかりませんが、このコードに説明を書いています。参考にでもしてください。

言語:Python,SQLite

使用した技術:スクレイピング（Webサイトから情報を取得してくる技術のこと）


プロセス：

処理に必要なデータのインポートおよびデータベースの作成

↓

各サイトからランキングを取得し、配列に書き込み

↓

配列からデータベースに情報を書き込み

↓

データベースにあるScoreの高い順にデータを出力

https://qiita.com/ku_a_i/items/9c68e8aed3c2c066bc60

In [ ]:
!pip install openpyxl #Excelデータの読み書き
!pip install mojimoji #全角文字を半角文字にするためのモジュール

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mojimoji: filename=mojimoji-0.0.12-cp310-cp310-linux_x86_64.whl size=173978 sha256=33346af7479a2f37995fbdc62269353b3fdff177b9f2e3997fd888450bbdc337
  Stored in directory: /root/.cache/pip/wheels/96/25/c5/225e7892b9b77f1b0b41b2fea7aaa34572e3f6012114e13100
Successfully built mojimoji


In [87]:
import sqlite3 #データベースを動かすためのモジュール
import pandas as pd #ランキングを作成するためのモジュール
import openpyxl #Excelから情報を取ってくるモジュール
import mojimoji #全角文字を半角文字に変換するためのモジュール

dbname = ('test.db')#データベース名.db拡張子で設定
conn = sqlite3.connect(dbname, isolation_level=None)#データベースを作成、自動コミット機能ON

In [88]:
cursor = conn.cursor() #カーソルオブジェクトを作成
sql = """CREATE TABLE music_master (
    Title TEXT,
    Artist TEXT,
    Score DOUBLE,
    Last_Rank INT,
    Last_Number INT,
    On_Chart INT,
    PRIMARY KEY (Title, Artist)
);"""
#↑DBのフォーマット設定（別のところに書いておきます）
cursor.execute(sql)#executeコマンドでSQL文を実行
conn.commit()#データベースにコミット(Excelでいう上書き保存。自動コミット設定なので不要だが一応・・)

In [89]:
sql = """INSERT INTO music_master VALUES(?,?,?,?,?,?)"""

data = [
   ('ラストノートしか知らない', '=LOVE', '', '14', '2167', '2'),
    ('ナツマトペ', '=LOVE', '', '', '', ''),
    ('ゲイシャフジヤマ', '#ババババンビ', '', '15', '2157', '1'),
    ('想わせぶりっこ', '≠ME', '', '17', '2160', '1'),
    ('唱', 'Ado', '', '5', '2168', '15'),
    ('クラクラ', 'Ado', '', '20', '2167', '9'),
    ('unravel', 'Ado', '', '13', '2164', '1'),
    ('オールナイトレディオ', 'Ado', '', '', '', ''),
    ('向日葵', 'Ado', '', '19', '2155', '7'),
    ('星の降る日に', 'aiko', '', '18', '2166', '2'),
    ('白色蜉蝣', 'Aimer', '', '9', '2167', '2'),
    ('アイドルなんかじゃなかったら', 'AKB48', '', '13', '2166', '7'),
    ('クリスマスソング', 'backnumber', '', '18', '2168', '1'),
    ('All I Ever Wanted feat.GULF KANAWUT', 'BALLISTIK BOYZ from EXILE TRIBE', '', '14', '2163', '4'),
    ('Glorius', 'BE:FIRST', '', '2', '2163', '1'),
    ('Mainstream', 'BE:FIRST', '', '4', '2157', '2'),
    ('Salvia', 'BE:FIRST', '', '', '', ''),
    ('Sleep Walking Orchestra', 'BUMP OF CHICKEN', '', '8', '2168', '1'),
    ('STARS', 'B\'z', '', '', '', ''),
    ('結-YOU-', 'ENHYPEN', '', '18', '2157', '4'),
    ('Tell Me', 'FANTASTICS from EXILE TRIBE', '', '', '', ''),
    ('わたしの一番かわいいところ', 'FRUITS ZIPPER', '', '11', '2155', '1'),
    ('DEAR MY LOVER', 'Hey! Say! JUMP', '', '20', '2161', '1'),
    ('乱心-RANSHIN-', 'Hiromitsu Kitayama', '', '11', '2165', '1'),
    ('SWITCHing', 'IMP.', '', '10', '2160', '1'),
    ('TAG ME(HANA_花)', 'INI', '', '18', '2165', '6'),
    ('Venus', 'JO1', '', '', '', ''),
    ('愛は勝つ', 'KAN', '', '15', '2164', '1'),
    ('My One Wish', 'Ken Yokoyama', '', '7', '2166', '2'),
    ('Wheelie', 'KID　PHENOMENON from EXILE TRIBE', '', '', '', ''),
    ('SPECIALZ', 'King Gnu', '', '6', '2168', '16'),
    ('硝子窓', 'King Gnu', '', '14', '2155', '1'),
    ('愛し生きること', 'King&Prince', '', '15', '2168', '4'),
    ('Perfect Night', 'LE SSERAFIM', '', '15', '2166', '1'),
    ('UNFORGIVEN（feat．NILE RODGERS）', 'LE SSERAFIM', '', '19', '2165', '1'),
    ('UNFORGIVEN(feat.NILE RODGERS)', 'LE SSERAFIM', '', '8', '2156', '8'),
    ('kimoto', 'Lienel', '', '13', '2162', '1'),
    ('今この瞬間を', 'Little Glee Monster', '', '8', '2165', '1'),
    ('UP TO ME！', 'Little Glee Monster', '', '', '', ''),
    ('spiral', 'LONGMAN', '', '', '', ''),
    ('Carnival', 'MAZZEL', '', '1', '2160', '1'),
    ('Anytime Anywhere', 'milet', '', '18', '2160', '4'),
    ('愛をありがとう', 'MISIA', '', '', '', ''),
    ('Fifty\'s map 〜おとなの地図', 'Mr.Children', '', '', '', ''),
    ('ケモノミチ', 'Mr.Children', '', '', '', ''),
    ('あなたに', 'Mrs.GREEN APPLE', '', '18', '2161', '1'),
    ('ANTENNA', 'Mrs.GREEN APPLE', '', '15', '2160', '2'),
    ('Magic', 'Mrs.GREEN APPLE', '', '', '', ''),
    ('ケセラセラ', 'Mrs.GREEN APPLE', '', '', '', ''),
    ('I’m a mess', 'MY FIRST STORY', '', '19', '2158', '6'),
    ('ギフテッド', 'NEWS', '', '14', '2166', '2'),
    ('あのさ、いや別に・・・', 'ＮＧＴ４８', '', '20', '2156', '1'),
    ('HEARTRIS', 'NiziU', '', '16', '2163', '2'),
    ('渚サイコー！', 'NMB48', '', '12', '2168', '4'),
    ('SOULSOUP', 'Official髭男dism', '', '10', '2168', '1'),
    ('Subtitle', 'Official髭男dism', '', '19', '2167', '30'),
    ('Chessboard', 'Official髭男dism', '', '7', '2155', '2'),
    ('日常', 'Official髭男dism', '', '10', '2156', '2'),
    ('MAKE It Out Alive', 'ONE OK ROCK', '', '', '', ''),
    ('飛んでゆけ', 'PEDRO', '', '', '', ''),
    ('Moon', 'Perfume', '', '', '', ''),
    ('切っ先', 'Reol', '', '', '', ''),
    ('最高到達点', 'SEKAI NO OWARI', '', '17', '2167', '9'),
    ('ターコイズ／サラバ／バタフライエフェクト', 'SEKAI NO OWARI', '', '', '', ''),
    ('人生遊戯', 'Sexy Zone', '', '2', '2168', '1'),
    ('本音と建前', 'Sexy Zone', '', '8', '2158', '3'),
    ('ＣＲＥＡＫ', 'SixTONE', '', '14', '2157', '5'),
    ('好きになっちゃった', 'SKE48', '', '11', '2159', '4'),
    ('Damagerholic', 'Snow Man', '', '15', '2167', '13'),
    ('タペストリー', 'Snow Man', '', '20', '2155', '10'),
    ('Social Path', 'Stray Kids', '', '', '', ''),
    ('君は何を後悔するのか', 'STU48', '', '14', '2168', '4'),
    ('Ashes', 'Superfly', '', '13', '2167', '3'),
    ('Now And Then', 'THE Beatles', '', '19', '2168', '4'),
    ('Jettin’', 'THE JET BOY BANGERS from EXILE TRIBE', '', '', '', ''),
    ('片隅', 'THE RAMPAGE from EXILE TRIBE', '', '4', '2163', '1'),
    ('Summer Riot ~熱帯夜~', 'THE RAMPAGE from EXILE TRIBE', '', '', '', ''),
    ('サマー☆★げっちゅー', 'THE SUPER FRUIT', '', '', '', ''),
    ('Whatever Comes', 'TM NETWORK', '', '', '', ''),
    ('JUST DANCE!', 'Travis Japan', '', '13', '2159', '1'),
    ('いけないfool logic', 'UNISON SQUARE GARDEN', '', '13', '2157', '1'),
    ('トドメの一撃(feat. Cory Wong)', 'Vaundy', '', '20', '2160', '2'),
    ('絶体絶命', 'WEST.', '', '18', '2163', '3'),
    ('Sweet Rain', 'WOLF HOWL HARMONEY from EXILE TRIBE', '', '', '', ''),
    ('アイドル', 'ＹＯＡＳＯＢＩ', '', '9', '2168', '37'),
    ('勇者', 'ＹＯＡＳＯＢＩ', '', '1', '2168', '12'),
    ('Biri-Biri', 'YOASOBI', '', '16', '2166', '3'),
    ('あのね', 'あいみょん', '', '13', '2168', '2'),
    ('愛の花', 'あいみょん', '', '15', '2158', '16'),
    ('ノット・オーケー', 'あいみょん', '', '17', '2158', '1'),
    ('RED LINE', 'アンジュルム', '', '7', '2168', '1'),
    ('うれしくて', 'いきものがかり', '', '', '', ''),
    ('With', '幾田りら', '', '14', '2158', '1'),
    ('No more cry', 'エレファントカシマシ', '', '8', '2161', '1'),
    ('ひかりのあとに', '上白石萌音', '', '17', '2161', '1'),
    ('Cross', '亀梨和也', '', '', '', ''),
    ('オオカミと彗星', '関ジャニ∞', '', '', '', ''),
    ('青のすみか', 'キタニタツヤ', '', '16', '2165', '19'),
    ('あいのロックンロール', 'ザ・クロマニヨンズ', '', '17', '2168', '1'),
    ('承認欲求', '櫻坂46', '', '', '', ''),
    ('Relay~杜の詩', 'サザンオールスターズ', '', '12', '2157', '2'),
    ('だってめぐり逢えたんだ', '純烈', '', '', '', ''),
    ('THE X(Dont\'Be Jealous/FLAVOR/Thank you)', 'ソ・イングク', '', '', '', ''),
    ('命日', 'ちゃんみな', '', '', '', ''),
    ('心音', '中島みゆき', '', '9', '2155', '1'),
    ('I Wish', 'なにわ男子', '', '11', '2168', '5'),
    ('Make Up Day', 'なにわ男子', '', 15, 2161, 6),
    ('more than words', '羊文字', '', 20, 2168, 5),
    ('Am I ready?', '日向坂46', '', '', '', ''),
    ('想望', '福山雅治', '', 16, 2168, 2),
    ('花', '藤井風', '', 16, 2167, 8),
    ('Workin’ Hard', '藤井風', '', '', '', ''),
    ('青空について考える', '僕が見たかった青空', '', '', '', ''),
    ('能動', '三浦大知', '', '', '', ''),
    ('スクラップアート', '水瀬いのり', '', 13, 2155, 1),
    ('すっごいFEVER！', 'モーニング娘。23', '', 3, 2161, 1),
    ('地球儀', '米津玄師', '', '', '', ''),
    ('月光浴', 'ヨルシカ', '', '', '', ''),
    ('花になって -Be a flower', '緑黄色社会', '', 4, 2168, 7),
    ('サマータイムシンデレラ', '緑黄色社会', '', 17, 2155, 7),
    ("Kissin' Christmas（クリスマスだからじゃない）", '桑田佳祐＆松任谷由実', '', 12, 2167, 2),
    ('Awakening', '三代目 J SOUL BROTHERS from EXILE TRIBE', '', 15, 2163, 1),
    ('るろうの形代', '菅田将暉, 東京スカパラダイスオーケストラ', '', 20, 2158, 1),
    ('あの空に立つ塔のように', '大泉洋', '', 20, 2162, 1),
    ('65535', '中川翔子', '', 17, 2162, 1),
    ('Monopoly', '乃木坂46', '', 3, 2168, 3),
    ('おひとりさま天国', '乃木坂46', '', 16, 2158, 5)
]
#↑過去のデータ
cursor.executemany(sql, data)#複数のデータを追加したい場合はexecutemanyメソッドを使う
conn.commit()#コミットする

In [103]:
import requests
from bs4 import BeautifulSoup #スクレイピング（取得）用
import datetime #日付計算用

#scoreを元にしたオリコンシングル・オリコンデジタル・ビルボード総合ランキング

# オリコン週間ランキング用
OriconWeekData = []
# オリコンデジタルランキング用
OriconDigitalData = []
# ビルボードランキング用
BillboardData = []
# 明屋書店ランキング用
HaruyaData = []


def selecter():

    print("見たい週を選択してください")
    select = int(input("1:今回 2:前回 3:別の回"))
    return select

def OriconTodays():
    # 今日の日付を求める
    d_today = datetime.date.today()
    # 今日の曜日を求める
    todayweek = datetime.date.today().weekday()
    #print(d_today, todayweek)

    #オリコンの発表は毎週水曜日のため、火曜日までは先週のランキングを表示
    #日付は来週の水曜日付となる。
    if (todayweek == 0):  # 今日が月曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today
    elif (todayweek == 1):# 今日が火曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today - datetime.timedelta(days=1)
    elif (todayweek == 2):# 今日が水曜日
        Oriconday = d_today + datetime.timedelta(days=5)
    elif (todayweek == 3):# 今日が木曜日
        Oriconday = d_today + datetime.timedelta(days=4)
    elif (todayweek == 4):# 今日が金曜日
        Oriconday = d_today + datetime.timedelta(days=3)
    elif (todayweek == 5):# 今日が土曜日
        Oriconday = d_today + datetime.timedelta(days=2)
    elif (todayweek == 6):# 今日が日曜日
        Oriconday = d_today + datetime.timedelta(days=1)

    return Oriconday


def OriconLastWeek():
    # 今日の日付を求める
    d_today = datetime.date.today()
    # 今日の曜日を求める
    todayweek = datetime.date.today().weekday()
    # print(d_today, todayweek)

    # オリコンの発表は毎週水曜日のため、火曜日までは先週のランキングを表示
    # 日付は来週の月曜日付となる。
    if (todayweek == 0):  # 今日が月曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today
    elif (todayweek == 1):  # 今日が火曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today - datetime.timedelta(days=1)
    elif (todayweek == 2):  # 今日が水曜日
        Oriconday = d_today - datetime.timedelta(days=2)
    elif (todayweek == 3):  # 今日が木曜日
        Oriconday = d_today - datetime.timedelta(days=3)
    elif (todayweek == 4):  # 今日が金曜日
        Oriconday = d_today - datetime.timedelta(days=4)
    elif (todayweek == 5):  # 今日が土曜日
        Oriconday = d_today - datetime.timedelta(days=5)
    elif (todayweek == 6):  # 今日が日曜日
        Oriconday = d_today - datetime.timedelta(days=6)

    return Oriconday


def OriconSelectWeek():
    # 日付の入力を促す
    date = input("2020年8月3日以降の日付を入力してください (YYYY-MM-DD): ")

    # 入力された日付をdatetimeオブジェクトに変換
    dt = datetime.datetime.strptime(date, "%Y-%m-%d")
    dt = dt.date()
    # 曜日を取得
    weekday = dt.weekday()

    # オリコンの発表は毎週水曜日のため、火曜日までは先週のランキングを表示
    # 日付は来週の月曜日付となる。
    if (weekday == 0):  # 今日が月曜日
        Oriconday = dt
    elif (weekday == 1):  # 今日が火曜日
        Oriconday = dt - datetime.timedelta(days=1)
    elif (weekday == 2):  # 今日が水曜日
        Oriconday = dt - datetime.timedelta(days=2)
    elif (weekday == 3):  # 今日が木曜日
        Oriconday = dt - datetime.timedelta(days=3)
    elif (weekday == 4):  # 今日が金曜日
        Oriconday = dt - datetime.timedelta(days=4)
    elif (weekday == 5):  # 今日が土曜日
        Oriconday = dt - datetime.timedelta(days=5)
    elif (weekday == 6):  # 今日が日曜日
        Oriconday = dt - datetime.timedelta(days=6)

    return Oriconday


def OriconWeekRank(Oriconday):#オリコン週間ランキング

    #1位から10位
    load_url = "https://www.oricon.co.jp/rank/js/w/" + str(Oriconday) + "/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2',class_='title') #曲名
    artist = soup.find(class_="content-rank-main").find_all('p',class_='name') #アーティスト名
    score = 6.0 #独自スコア
    rank = 1 #ランキング
    print(str(Oriconday) + "付けオリコン週間シングルランキング")


    for link, artist in zip(links, artist):
      # 10位までランクインしたものを配列に保存
      OriconWeekData.append([link.text,artist.text,"{:.1f}".format(score)])


      # 壊れたときの表示用
      # if rank != 10:#10位以下（１ケタの場合）なら（点数の位置を揃えるため）
      #   print(" " + str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
      # else: #10位（２ケタの場合）なら
      #   print(str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
      rank = rank + 1
      score = score - 0.3


    #11位から20位
    load_url = "https://www.oricon.co.jp/rank/js/w/" + str(Oriconday) + "/p/2/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2', class_='title')  # 曲名
    artist = soup.find(class_="content-rank-main").find_all('p', class_='name')  # アーティスト名
    for link, artist in zip(links, artist):
      OriconWeekData.append([link.text,artist.text,"{:.1f}".format(score)])
        # 壊れたときの表示用
      # print(str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
      rank = rank + 1
      score = score - 0.3

    # print(OriconWeekRank)



def OriconDigitalRank(Oriconday):#オリコンデジタルシングルランキング
    # 1位から10位
    load_url = "https://www.oricon.co.jp/rank/dis/w/" + str(Oriconday) + "/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2', class_='title')
    artist = soup.find(class_="content-rank-main").find_all('p', class_='name')  # アーティスト名
    rank = 1
    score = float(6.0)
    print(str(Oriconday) + "付けオリコン週間デジタルシングルランキング")
    for link, artist in zip(links, artist):
      # 壊れたときの表示用
        # if rank != 10:#10位以下（１ケタの場合）なら（点数の位置を揃えるため）
        #     print(" " + str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
        # else: #10位（２ケタの場合）なら
        #     print(str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
        OriconDigitalData.append([link.text,artist.text,"{:.1f}".format(score)])
        rank = rank + 1
        score = score - 0.3

    # 11位から20位
    load_url = "https://www.oricon.co.jp/rank/dis/w/" + str(Oriconday) + "/p/2/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2', class_='title')
    artist = soup.find(class_="content-rank-main").find_all('p', class_='name')  # アーティスト名
    for link, artist in zip(links, artist):
        OriconDigitalData.append([link.text,artist.text,"{:.1f}".format(score)])
        # print(str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
        rank = rank + 1
        score = score - 0.3

def BillboadRank(Oriconday):#ビルボードJAPAN HOT100ランキング

    # オリコンの日付とビルボードの発表日の差を埋めるための計算
    Billday = Oriconday - datetime.timedelta(days=5)

    print(str(Billday) + "付けビルボードJAPAN HOT100ランキング")

    #URL(ここを変更すると読み込まなくなります)
    url = 'https://www.billboard-japan.com/charts/detail?a=hot100&year='+str(Oriconday.year)+'&month='+str(Oriconday.month)+'&day='+str(Oriconday.day)
    #URLを取得してくる
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    songs = soup.find_all('p', class_='musuc_title') #曲名
    artists = soup.find_all('p', class_='artist_name') #アーティスト名
    score = 6.0 #基準点

    for i in range(20):#20回繰り替えす
      song = songs[i].text.strip()
      artist = artists[i].text.strip()
      BillboardData.append([song,artist,format(score, '.1f')])
      # if i < 9:
      #   print(f" {i+1}位: {format(score, '.1f')} {song} / {artist}") #1位から9位までのランキング
      # else:
      #   print(f"{i+1}位: {format(score, '.1f')} {song} / {artist}") #10位から20位までのランキング
      score = score - 0.3 #scoreを-0.3する


sel = selecter()
print(sel)

if sel==1: #今週のランキング表示
    OriconTodays()
    OriconWeekRank(OriconTodays())
    print()
    OriconDigitalRank(OriconTodays())
    print()
    BillboadRank(OriconTodays())

elif sel == 2: #前週のランキング表示
    OriconLastWeek()
    OriconWeekRank(OriconLastWeek())
    # print(OriconWeekData)
    print()
    OriconDigitalRank(OriconLastWeek())
    # print(OriconDigitalData)
    print()
    BillboadRank(OriconLastWeek())
    # print(BillboardData)

else : #別の週のランキング
    OriconWeekRank(OriconSelectWeek())
    print()
    OriconDigitalRank(OriconSelectWeek())
    print()
    BillboadRank(OriconSelectWeek())


見たい週を選択してください
1:今回 2:前回 3:別の回3
3
2020年8月3日以降の日付を入力してください (YYYY-MM-DD): 2023-10-02
2023-10-02付けオリコン週間シングルランキング

2020年8月3日以降の日付を入力してください (YYYY-MM-DD): 2023-10-02
2023-10-02付けオリコン週間デジタルシングルランキング

2020年8月3日以降の日付を入力してください (YYYY-MM-DD): 2023-10-02
2023-09-27付けビルボードJAPAN HOT100ランキング


In [104]:
#すでに存在するExcelワークブックを読み込む(絶対パス)
wb = openpyxl.load_workbook("1002ベストヒット愛媛.xlsx")
#ワークブック内のアクティブなシートを取得
ws = wb.active

HaruyaData = [[mojimoji.zen_to_han(ws["D4"].value, kana=False),mojimoji.zen_to_han(ws["C4"].value, kana=False),6.0],
              [mojimoji.zen_to_han(ws["D5"].value, kana=False),mojimoji.zen_to_han(ws["C5"].value, kana=False),5.7],
              [mojimoji.zen_to_han(ws["D6"].value, kana=False),mojimoji.zen_to_han(ws["C6"].value, kana=False),5.4],
              [mojimoji.zen_to_han(ws["D7"].value, kana=False),mojimoji.zen_to_han(ws["C7"].value, kana=False),5.1],
              [mojimoji.zen_to_han(ws["D8"].value, kana=False),mojimoji.zen_to_han(ws["C8"].value, kana=False),4.8],
              [mojimoji.zen_to_han(ws["D9"].value, kana=False),mojimoji.zen_to_han(ws["C9"].value, kana=False),4.5],
              [mojimoji.zen_to_han(ws["D10"].value, kana=False),mojimoji.zen_to_han(ws["C10"].value, kana=False),4.2],
              [mojimoji.zen_to_han(ws["D11"].value, kana=False),mojimoji.zen_to_han(ws["C11"].value, kana=False),3.9],
              [mojimoji.zen_to_han(ws["D12"].value, kana=False),mojimoji.zen_to_han(ws["C12"].value, kana=False),3.6],
              [mojimoji.zen_to_han(ws["D13"].value, kana=False),mojimoji.zen_to_han(ws["C13"].value, kana=False),3.3],
              [mojimoji.zen_to_han(ws["D14"].value, kana=False),mojimoji.zen_to_han(ws["C14"].value, kana=False),3.0],
              [mojimoji.zen_to_han(ws["D15"].value, kana=False),mojimoji.zen_to_han(ws["C15"].value, kana=False),2.7],
              [mojimoji.zen_to_han(ws["D16"].value, kana=False),mojimoji.zen_to_han(ws["C16"].value, kana=False),2.4],
              [mojimoji.zen_to_han(ws["D17"].value, kana=False),mojimoji.zen_to_han(ws["C17"].value, kana=False),2.1],
              [mojimoji.zen_to_han(ws["D18"].value, kana=False),mojimoji.zen_to_han(ws["C18"].value, kana=False),1.8],
              [mojimoji.zen_to_han(ws["D19"].value, kana=False),mojimoji.zen_to_han(ws["C19"].value, kana=False),1.5],
              [mojimoji.zen_to_han(ws["D20"].value, kana=False),mojimoji.zen_to_han(ws["C20"].value, kana=False),1.2],
              [mojimoji.zen_to_han(ws["D21"].value, kana=False),mojimoji.zen_to_han(ws["C21"].value, kana=False),0.9],
              [mojimoji.zen_to_han(ws["D22"].value, kana=False),mojimoji.zen_to_han(ws["C22"].value, kana=False),0.6],
              [mojimoji.zen_to_han(ws["D23"].value, kana=False),mojimoji.zen_to_han(ws["C23"].value, kana=False),0.3],
              ]


print(HaruyaData)

[['Dangerholic', 'Snow Man', 6.0], ['Make Up Day/Missing', 'なにわ男子', 5.7], ['アイドルなんかじゃなかったら', 'AKB48', 5.4], ['本音と建前', 'Sexy Zone', 5.1], ['SPECIALZ', 'King Gnu', 4.8], ['CREAK', 'SixTONES', 4.5], ['愛の花', 'あいみょん', 4.2], ['オレンジkiss', 'Snow Man', 3.9], ['春待ち鳥よ・・・', '岡ゆう子', 3.6], ['Mainstream', 'BE:FIRST', 3.3], ['哀傷歌/風を起こせ', '鳥羽一郎', 3.0], ['結-YOU-', 'ENHYPEN', 2.7], ['ブラザービート', 'Snow Man', 2.4], ['釧網本線', '走裕介', 2.1], ['鏡の花', '服部浩子', 1.8], ['海峡浪漫', '山内惠介', 1.5], ['心音(しんおん)', '中島みゆき', 1.2], ['UNFORGIVEN', 'LE SSERAFIM', 0.9], ['こっから', 'SixTONES', 0.6], ['THE IDOLM@STER MILLION THE@TER VARIETY 04', '我那覇響,横山奈緒,高坂海美,真壁瑞希,宮尾美也,三浦あずさ,二階堂千鶴,四条貴音,北上麗花,豊川風', 0.3]]


In [105]:
# 二次元配列にある情報をデータベースに登録
# オリコン週間ランキング
for entry in OriconWeekData:
    title, artist, score = entry
    # 既存のデータがあればScoreを足して更新、なければ新規追加
    cursor.execute('''
        INSERT INTO music_master (Title, Artist, Score, Last_Rank, Last_Number, On_Chart)
        VALUES (?, ?, ?, 0, 0, 0)
        ON CONFLICT(Title, Artist) DO UPDATE SET Score = Score + ?
    ''', (title, artist, score, score))

# コミットして変更を保存
conn.commit()

In [106]:
# オリコンデジタルランキング
for entry in OriconDigitalData:
    title, artist, score = entry
    # 既存のデータがあればScoreを足して更新、なければ新規追加
    cursor.execute('''
        INSERT INTO music_master (Title, Artist, Score, Last_Rank, Last_Number, On_Chart)
        VALUES (?, ?, ?, 0, 0, 0)
        ON CONFLICT(Title, Artist) DO UPDATE SET Score = Score + ?
    ''', (title, artist, score, score))

# コミットして変更を保存
conn.commit()

In [107]:
# ビルボードJAPAN
for entry in BillboardData:
    title, artist, score = entry
    # 既存のデータがあればScoreを足して更新、なければ新規追加
    cursor.execute('''
        INSERT INTO music_master (Title, Artist, Score,Last_Rank, Last_Number, On_Chart)
        VALUES (?, ?, ?, 0, 0, 0)
        ON CONFLICT(Title, Artist) DO UPDATE SET Score = Score + ?
    ''', (title, artist, score, score))

# コミットして変更を保存
conn.commit()

In [108]:
# 明屋書店
for entry in HaruyaData:
    title, artist, score = entry
    # 既存のデータがあればScoreを足して更新、なければ新規追加
    cursor.execute('''
        INSERT INTO music_master (Title, Artist, Score, Last_Rank, Last_Number, On_Chart)
        VALUES (?, ?, ?, 0, 0, 0)
        ON CONFLICT(Title, Artist) DO UPDATE SET Score = Score + ?
    ''', (title, artist, score, score))

# コミットして変更を保存
conn.commit()

**データベースから上位20個を出力する**

整形後もここの再生ボタンで出力してください

In [109]:
# Scoreの高い順に上位25件を取得
top_20_query = '''
    SELECT * FROM music_master
    WHERE Score IS NOT NULL AND Score != ''
    ORDER BY Score DESC
    LIMIT 20
'''
top_20_results = cursor.execute(top_20_query).fetchall()
# コミットして変更を保存
conn.commit()

# 整形後のデータ出力用
df = pd.read_sql('''
SELECT * FROM music_master
WHERE Score IS NOT NULL AND Score != ''
ORDER BY Score DESC
LIMIT 60''', conn)

df.head(20)


,Title,Artist,Score,Last_Rank,Last_Number,On_Chart
0,本音と建前,Sexy Zone,27.0,8,2158,3
1,Mainstream,BE:FIRST,26.1,4,2157,2
2,SPECIALZ,King Gnu,24.6,6,2168,16
3,Make Up Day/Missing,なにわ男子,22.2,0,0,0
4,Dangerholic,Snow Man,22.2,0,0,0
5,唱,Ado,16.8,5,2168,15
6,IMP.,IMP.,14.1,0,0,0
7,アイドル,YOASOBI,13.5,0,0,0
8,Relay~杜の詩,サザンオールスターズ,12.0,12,2157,2
9,最高到達点,SEKAI NO OWARI,11.4,17,2167,9


**==========================ここまで=================================**

**ここから下はデータ整形用**

%はFM愛媛で言うレコード室の検索の＊と同じです

**指定したタイトルの行を削除する**

In [81]:
# 削除したいTitleを入力してください（%は消さないでください（前方一致検索））
cursor.execute('''DELETE FROM music_master
WHERE Title LIKE
 'ジェネレーション・ギャップ%';
 ''')


**指定したタイトルのScoreを任意の点数追加する**

In [ ]:
# 削除したいTitleとScoreの値を入力してください（%は消さないでください（前方一致検索））
cursor.execute('''UPDATE music_master
SET Score = Score + （ここに点数を半角数字で入力）
WHERE Title LIKE
'（指定したタイトル）%';
 ''')

**指定したタイトルのScoreを+3点する**

In [ ]:
# 3点追加したいTitleを入力してください（%は消さないでください（前方一致検索））
cursor.execute('''UPDATE music_master
SET Score = Score + 3
WHERE Title LIKE
'（指定したタイトル）%';
 ''')

**データベースに曲名・アーティスト名・Scoreを書き込む**

明屋書店のExcel用（既にデータベースに曲がある場合は、上の任意の点数入力を用いること）

In [ ]:
# 一番下の行に必要事項を入力してください
cursor.execute('''
        INSERT INTO music_master (Title, Artist, Score, Last_Rank, Last_Number, On_Chart)
        VALUES (?, ?, ?, 0, 0, 0)
    ''', ('（曲名）', '（アーティスト名）', '（Scoreの値）'))

**データベース検索**

整備中

In [ ]:
cursor.execute('''
SELECT * FROM music_master
WHERE Title LIKE 'わたしの%'
   OR Artist LIKE 'F%';
    ''')



**マジカルサイト送信用Excel自動生成**

In [110]:
from openpyxl.styles import Font, PatternFill
from openpyxl import load_workbook

# Excelファイルを読み込み
workbook = load_workbook('ベストヒットランキング テスト1.xlsx')
sheet = workbook.active

# sheet.unmerge_cells('A1:F25')

# This_Numberの値を設定
print("今回のベストヒットランキングの回数を入力してください")
this_number = int(input())  # 例として100を設定（実際の値に置き換えてください）

sheet.cell(row=3, column=2).value = "Ｎｏ."+str(+this_number) #No.
sheet.cell(row=3, column=6).value = str(OriconTodays().year)+"年"+str(OriconTodays().month)+"月"+str(OriconTodays().day)+"日"#日付
# セルにデータを書き込み
for idx, result in enumerate(top_20_results, start=1):
    title, artist, score, last_rank, last_number, on_chart= result

    # Last_Rankが空文字もしくはnullの場合、文字色を赤に変更
    if last_rank is None or last_rank == "" or last_rank == 0:
        sheet.cell(row=idx+5, column=2).value = idx
        sheet.cell(row=idx+5, column=2).font = Font(name ="BIZ UDPゴシック",size = 16,bold=True,color="FF0000") #ThisWeek
        sheet.cell(row=idx+5, column=3).value = "初"
        sheet.cell(row=idx+5, column=3).font = Font(name ="BIZ UDPゴシック",size = 16,bold=True,color="FF0000") #LastWeek
        sheet.cell(row=idx+5, column=4).value = "1"
        sheet.cell(row=idx+5, column=4).font = Font(name ="BIZ UDPゴシック",size = 16,bold=True,color="FF0000") #Onchart
        sheet.cell(row=idx+5, column=5).value = title
        sheet.cell(row=idx+5, column=6).value = artist

    # Last_NumberがThis_Numberより2以上差がある場合、セルに「再」を入力
    elif (last_number - this_number) >= 2:
        sheet.cell(row=idx+5, column=2).value = idx#ThisWeek
        sheet.cell(row=idx+5, column=3).value = "再"#LastWeek
        sheet.cell(row=idx+5, column=4).value = on_chart+1
        sheet.cell(row=idx+5, column=5).value = title
        sheet.cell(row=idx+5, column=6).value = artist

    else:
    # セルにデータを書き込み
        sheet.cell(row=idx+5, column=5).value = title
        sheet.cell(row=idx+5, column=6).value = artist
        sheet.cell(row=idx+5, column=2).value = idx
        sheet.cell(row=idx+5, column=3).value = last_rank
        # sheet.cell(row=idx, column=7).value = last_number
        sheet.cell(row=idx+5, column=4).value = on_chart+1

# Excelファイルを保存
workbook.save('ベストヒットランキング テスト２.xlsx')

# データベース接続を閉じる
# conn.close()

今回のベストヒットランキングの回数を入力してください
2156


**終わったら上の出力を押してください**

**終了後はここから下を実行**

必ずランキングが作成されたことを確認すること！

In [114]:
  # 今回のベストヒットランキングの回数を入力

for idx, result in enumerate(top_20_results, start=1):
    title, artist, score, _, _, _ = result
    cursor.execute('''
        UPDATE music_master
        SET Last_Rank = ?, On_Chart = On_Chart + 1, Last_Number = ?
        WHERE Title = ? AND Artist = ?
    ''', (idx, this_number, title, artist))

cursor.execute('UPDATE music_master SET Score = 0;') #Scoreの値を全消し

In [115]:
df = pd.read_sql('''SELECT * FROM music_master ;''',conn)
df.head(60)

,Title,Artist,Score,Last_Rank,Last_Number,On_Chart
0,ラストノートしか知らない,=LOVE,0.0,14,2167,2
1,ナツマトペ,=LOVE,0.0,,,
2,ゲイシャフジヤマ,#ババババンビ,0.0,15,2157,1
3,想わせぶりっこ,≠ME,0.0,17,2160,1
4,唱,Ado,0.0,6,2156,17
5,クラクラ,Ado,0.0,20,2167,9
6,unravel,Ado,0.0,13,2164,1
7,オールナイトレディオ,Ado,0.0,,,
8,向日葵,Ado,0.0,19,2155,7
9,星の降る日に,aiko,0.0,18,2166,2


In [ ]:
print()